# Modeling for Snow Depth Prediction

This notebook is focused on preparing the data for modeling, addressing multicollinearity, and creating training, validation, and testing sets. We will proceed with the following steps:
- Loading the Processed Data
- Creating Lag Features and Derived Variables
- Handling Multicollinearity
- Splitting the Data into Training, Validation, and Testing Sets

### 1. Loading Libraries and Processed Data

In [ ]:
import pandas as pd
import numpy as np
import os
from statsmodels.stats.outliers_influence import variance_inflation_factor

### 2. Load Processed Data

In [1]:
processed_data_root = 'data/processed/cds'
# Initialize a dictionary to store DataFrames for each resort
processed_resort_data = {}

csv_files = glob.glob(os.path.join(processed_data_root, '**', '*_for_modeling.csv'), recursive=True)

for file_path in csv_files:
    df = pd.read_csv(file_path)
    resort_name = os.path.splitext(os.path.basename(file_path))[0]
    processed_resort_data[resort_name] = df
    print(f"Loaded processed data for {resort_name} with shape {df.shape}")

NameError: name 'glob' is not defined